<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from tensorflow import keras as K
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import os

In [2]:
tf.__version__

'2.5.0'

In [3]:
data=os.path.join(os.path.dirname('.'),'data/nlp/timemachine.txt')
#data=os.path.join(os.path.dirname('.'),'data/nlp/tinyshakespeare.txt')
text=open(data,'r',encoding='utf-8').read()

In [4]:
# Take a look at the first 200 characters in text
print(text[:200])

Project Gutenberg's The Time Machine, by H. G. (Herbert George) Wells

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it awa


In [5]:
def preprocess(text1):
    lower=text.lower()
    chars_to_rem = ['\n','#', '*',  '_','/', '"', '$', '%', "'", '(', ')', ',', '-','[', ']']
    for i in chars_to_rem:
        lower=lower.replace(i,'')
    return lower
text=preprocess(text)

In [6]:
print(f' The Lenght of Corpus is : {len(text)}')

 The Lenght of Corpus is : 210109


In [7]:
chars=sorted(list(set(text)))
n_chars=len(chars)
char_to_idx=dict((c,i) for i,c in enumerate(chars))
idx_to_char=dict((i,c) for i,c in enumerate(chars))

In [8]:
print(chars),print(n_chars)

[' ', '!', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
43


(None, None)

In [9]:
steps=1
seq_len=50
sentences=list()
next_char=list()
for i in range(0,len(text)-seq_len,steps):
    sentences.append(text[i:i+seq_len])
    next_char.append(text[i+seq_len])
print(f'Number of sequences :{len(sentences)}' )

Number of sequences :210059


In [10]:
x=np.zeros((len(sentences),seq_len,n_chars),dtype=np.bool)
y=np.zeros((len(sentences),n_chars),dtype=np.bool)
for i,sent in enumerate(sentences):
    for j, char in enumerate(sent):
        x[i,j,char_to_idx[char]]=1
    y[i,char_to_idx[next_char[i]]]=1

In [11]:
model=K.Sequential([
    K.layers.InputLayer(input_shape=(seq_len,n_chars)),
    K.layers.LSTM(100),
    K.layers.Dropout(0.01),
    K.layers.Dense(n_chars, activation="softmax") 
       ])
model.compile(loss="categorical_crossentropy", optimizer='rmsprop')

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               57600     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 43)                4343      
Total params: 61,943
Trainable params: 61,943
Non-trainable params: 0
_________________________________________________________________


In [13]:
epochs = 40
batch_size = 128
model.fit(x, y, batch_size=batch_size, epochs=epochs)

Epoch 1/40
1642/1642 [==============================] - 176s 106ms/step - loss: 2.2671
Epoch 2/40
1642/1642 [==============================] - 168s 102ms/step - loss: 1.9289
Epoch 3/40
1642/1642 [==============================] - 170s 104ms/step - loss: 1.7965
Epoch 4/40
1642/1642 [==============================] - 169s 103ms/step - loss: 1.7106
Epoch 5/40
1642/1642 [==============================] - 165s 100ms/step - loss: 1.6461
Epoch 6/40
1642/1642 [==============================] - 164s 100ms/step - loss: 1.5935
Epoch 7/40
1642/1642 [==============================] - 164s 100ms/step - loss: 1.5514
Epoch 8/40
1642/1642 [==============================] - 169s 103ms/step - loss: 1.5165
Epoch 9/40
1642/1642 [==============================] - 164s 100ms/step - loss: 1.4857
Epoch 10/40
1642/1642 [==============================] - 164s 100ms/step - loss: 1.4597
Epoch 11/40
1642/1642 [==============================] - 164s 100ms/step - loss: 1.4368
Epoch 12/40
1642/1642 [==================

In [14]:
model.save('generated_model')

INFO:tensorflow:Assets written to: generated_model\assets


INFO:tensorflow:Assets written to: generated_model\assets


In [15]:
def sample(preds, temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    preds=np.exp(preds)
    preds=preds/np.sum(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [16]:
import random

In [17]:
start_index = random.randint(0, len(text)-seq_len - 1)
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('Temperature',temperature)
    generated = ''
    sentence = text[start_index:start_index+seq_len]
    print('...Generating with seed: "' + sentence + '"')
    for i in range(300):
        x_pred=np.zeros((1,seq_len,n_chars))
        for t,char in enumerate(sentence):
            x_pred[0,t,char_to_idx[char]]=1.0
        pred=model.predict(x_pred,verbose=0)[0]
        next_index = sample(pred, temperature)
        next_char = idx_to_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char
        print("...Generated: ", generated)
        print()
    
    

Temperature 0.2
...Generating with seed: "ng shafts and wells along the hill slopeseverywher"
...Generated:  e

...Generated:  e 

...Generated:  e i

...Generated:  e in

...Generated:  e in 

...Generated:  e in t

...Generated:  e in th

...Generated:  e in the

...Generated:  e in the 

...Generated:  e in the s

...Generated:  e in the su

...Generated:  e in the sun

...Generated:  e in the suns

...Generated:  e in the sunse

...Generated:  e in the sunset

...Generated:  e in the sunset 

...Generated:  e in the sunset o

...Generated:  e in the sunset of

...Generated:  e in the sunset of 

...Generated:  e in the sunset of t

...Generated:  e in the sunset of th

...Generated:  e in the sunset of the

...Generated:  e in the sunset of the 

...Generated:  e in the sunset of the r

...Generated:  e in the sunset of the re

...Generated:  e in the sunset of the rea

...Generated:  e in the sunset of the real

...Generated:  e in the sunset of the real 

...Generated:  e in the 

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by p

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by pr

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by pro

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by proj

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by proje

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by projec

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project

...Generated:  e in the sunset of the real and the present down and the same

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gut

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gute

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project guten

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenb

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenbe

...Generated:  e in the sunset of the real and the present down a

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenberg literary archive foundation of the pro

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenberg literary archive foundation of the proj

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenberg literary archive foundation of the proje

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenberg literary archive foundation of the projec

...Generated:  e in the sunset of the real and the present down and the sa

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenberg literary archive foundation of the project gutenbergtm works be surrand and 

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenberg literary archive foundation of the project gutenbergtm works be surrand and t

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenberg literary archive foundation of the project gutenbergtm works be surrand and th

...Generated:  e in the sunset of the real and the present down and the same concribations of the project gutenbergtm work by project gutenbergtm work in a remive with the project gutenberg lit

...

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxi

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxit

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxita

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitat

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitati

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof 

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyyl

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyyla

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylal

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylali

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum r

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylalithines toparrat wseelling oner handso

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylalithines toparrat wseelling oner handsof

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylalithines toparrat wseelling oner handsoft

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylalithines topar

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylalithines toparrat wseelling oner handsoft. sockeliby scan ofmhat istoni

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylalithines toparrat wseelling oner handsoft. sockeliby scan ofmhat istonig

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylalithines toparrat wseelling oner handsoft. sockeliby scan ofmhat istonigh

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum reco

...Generated:  e evise of the evendidy ofdotch wat ebilly anvatings these setbroni1y under and when i cannother with little caum recondetof  the outxuphed upon the nuxitation abclaenly face envered usangpredoullyylalithines toparrat wseelling oner handsoft. sockeliby scan ofmhat istonighs by they.up all ant impred

